In [1]:
import os
import sys
import pandas as pd
import openpyxl
import json
from datetime import datetime, timedelta
from dateutil import parser as datetime_parser
from functions import *

In [2]:
# Read downloaded Grow data
grow_file_name = "enterprise-mqls-trend-totals-2022-04-13-08_00.xlsx"

grow_df = pd.read_excel(grow_file_name)

grow_df.shape

(28519, 9)

In [3]:
# Read in hubspot data
hs_contacts = pd.read_csv("hubspot_contacts.csv")
hs_contacts.head()

,id,properties,propertiesWithHistory,createdAt,updatedAt,archived,campaign___paid,campaign_source,createdate,hs_analytics_source,hs_analytics_source_data_1,hs_analytics_source_data_2,hs_object_id,lastmodifieddate,lead_source,salesforcecontactid,salesforceleadid
0,5551,"{'campaign___paid': 'spotahome_hyper_growth', ...",{'campaign___paid': [{'value': 'spotahome_hype...,2017-09-26T08:28:27.856Z,2022-04-12T10:35:09.307Z,False,spotahome_hyper_growth,linkedin / impressions,2017-09-26T08:28:27.856Z,OTHER_CAMPAIGNS,spotahome_hyper_growth,linkedin / impressions,5551,2022-04-12T10:35:09.307Z,Paid Campaign,NaN,00Q4W00001ROq8FUAT
1,5601,"{'campaign___paid': 'spotahome_hyper_growth', ...",{'campaign___paid': [{'value': 'spotahome_hype...,2017-09-26T09:13:41.779Z,2022-04-12T10:35:08.252Z,False,spotahome_hyper_growth,linkedin / impressions,2017-09-26T09:13:41.779Z,OTHER_CAMPAIGNS,spotahome_hyper_growth,linkedin / impressions,5601,2022-04-12T10:35:08.252Z,Paid Campaign,NaN,00Q4W00001RP4oEUAT
2,10851,"{'campaign___paid': 'spotahome_hyper_growth', ...",{'campaign___paid': [{'value': 'spotahome_hype...,2017-09-27T14:45:36.035Z,2022-02-06T18:42:47.198Z,False,spotahome_hyper_growth,linkedin / impressions,2017-09-27T14:45:36.035Z,OTHER_CAMPAIGNS,spotahome_hyper_growth,linkedin / impressions,10851,2022-02-06T18:42:47.198Z,Other,NaN,NaN
3,22551,"{'campaign___paid': 'spotahome_hyper_growth', ...",{'campaign___paid': [{'value': 'spotahome_hype...,2017-09-29T14:18:44.589Z,2022-04-12T10:35:07.355Z,False,spotahome_hyper_growth,linkedin / impressions,2017-09-29T14:18:44.589Z,OTHER_CAMPAIGNS,spotahome_hyper_growth,linkedin / impressions,22551,2022-04-12T10:35:07.355Z,Other,0034W00002NjFl5QAF,NaN
4,100801,"{'campaign___paid': 'goodlord case study', 'ca...",{'campaign___paid': [{'value': 'goodlord case ...,2017-10-19T22:36:29.019Z,2022-04-12T10:35:07.576Z,False,goodlord case study,Google,2017-10-19T22:36:29.019Z,PAID_SEARCH,goodlord case study,goal setting,100801,2022-04-12T10:35:07.576Z,Paid Campaign,NaN,00Q4W00001RP3sPUAT


In [4]:
# Take each campaign__paid in propertiesWithHistory and create a new row based on it
hs_contact_campaign = []
count = 0

for index, row in hs_contacts.iterrows():
    # print(row)

    properties_with_history = row["propertiesWithHistory"].replace("'", '"')

    history = json.loads(properties_with_history)

    for campaign_paid, campaign_source in zip(history["campaign___paid"], history["campaign_source"]):
        # print(item)
        existing_row_data = {
            header: row[header]
            for header in row.index
            if "properties" not in header
        }
        
        # Add prefixes to avoid naming collusions
        paid = {f"campaign_paid.{key}": value for key, value in campaign_paid.items()}
        source = {f"campaign_source.{key}": value for key, value in campaign_source.items()}

        hs_contact_campaign.append({**existing_row_data, **paid, ** source})

hs_contact_campaign_df = pd.DataFrame(hs_contact_campaign)
hs_contact_campaign_df.head()

,id,createdAt,updatedAt,archived,campaign___paid,campaign_source,createdate,hs_analytics_source,hs_analytics_source_data_1,hs_analytics_source_data_2,...,salesforcecontactid,salesforceleadid,campaign_paid.value,campaign_paid.timestamp,campaign_paid.sourceType,campaign_paid.sourceId,campaign_source.value,campaign_source.timestamp,campaign_source.sourceType,campaign_source.sourceId
0,5551,2017-09-26T08:28:27.856Z,2022-04-12T10:35:09.307Z,False,spotahome_hyper_growth,linkedin / impressions,2017-09-26T08:28:27.856Z,OTHER_CAMPAIGNS,spotahome_hyper_growth,linkedin / impressions,...,NaN,00Q4W00001ROq8FUAT,spotahome_hyper_growth,2021-04-29T09:39:38.109Z,AUTOMATION_PLATFORM,enrollmentId:101078377809;actionExecutionIndex:2,linkedin / impressions,2021-04-29T09:39:37.188Z,AUTOMATION_PLATFORM,enrollmentId:101078377809;actionExecutionIndex:1
1,5601,2017-09-26T09:13:41.779Z,2022-04-12T10:35:08.252Z,False,spotahome_hyper_growth,linkedin / impressions,2017-09-26T09:13:41.779Z,OTHER_CAMPAIGNS,spotahome_hyper_growth,linkedin / impressions,...,NaN,00Q4W00001RP4oEUAT,spotahome_hyper_growth,2021-04-29T09:39:38.994Z,AUTOMATION_PLATFORM,enrollmentId:101078377852;actionExecutionIndex:2,linkedin / impressions,2021-04-29T09:39:38.154Z,AUTOMATION_PLATFORM,enrollmentId:101078377852;actionExecutionIndex:1
2,10851,2017-09-27T14:45:36.035Z,2022-02-06T18:42:47.198Z,False,spotahome_hyper_growth,linkedin / impressions,2017-09-27T14:45:36.035Z,OTHER_CAMPAIGNS,spotahome_hyper_growth,linkedin / impressions,...,NaN,NaN,spotahome_hyper_growth,2021-04-29T09:39:39.139Z,AUTOMATION_PLATFORM,enrollmentId:101077198159;actionExecutionIndex:2,linkedin / impressions,2021-04-29T09:39:38.078Z,AUTOMATION_PLATFORM,enrollmentId:101077198159;actionExecutionIndex:1
3,22551,2017-09-29T14:18:44.589Z,2022-04-12T10:35:07.355Z,False,spotahome_hyper_growth,linkedin / impressions,2017-09-29T14:18:44.589Z,OTHER_CAMPAIGNS,spotahome_hyper_growth,linkedin / impressions,...,0034W00002NjFl5QAF,NaN,spotahome_hyper_growth,2021-04-29T09:39:38.446Z,AUTOMATION_PLATFORM,enrollmentId:101074311528;actionExecutionIndex:2,linkedin / impressions,2021-04-29T09:39:37.184Z,AUTOMATION_PLATFORM,enrollmentId:101074311528;actionExecutionIndex:1
4,100801,2017-10-19T22:36:29.019Z,2022-04-12T10:35:07.576Z,False,goodlord case study,Google,2017-10-19T22:36:29.019Z,PAID_SEARCH,goodlord case study,goal setting,...,NaN,00Q4W00001RP3sPUAT,goodlord case study,2021-04-29T09:39:39.018Z,AUTOMATION_PLATFORM,enrollmentId:101078897924;actionExecutionIndex:3,Google,2021-04-29T09:39:38.397Z,AUTOMATION_PLATFORM,enrollmentId:101078897924;actionExecutionIndex:2


In [5]:
# Fill null values
hs_contact_campaign_df["salesforcecontactid"] = hs_contact_campaign_df["salesforcecontactid"].fillna("")
hs_contact_campaign_df["salesforceleadid"] = hs_contact_campaign_df["salesforceleadid"].fillna("")

# Create a new column to handle the Salesforce join
hs_contact_campaign_df["salesforce_id"] = hs_contact_campaign_df[["salesforcecontactid", "salesforceleadid"]].T.agg("".join)

In [6]:
hs_contact_campaign_df[["salesforceleadid", "salesforcecontactid", "salesforce_id"]].head()

,salesforceleadid,salesforcecontactid,salesforce_id
0,00Q4W00001ROq8FUAT,,00Q4W00001ROq8FUAT
1,00Q4W00001RP4oEUAT,,00Q4W00001RP4oEUAT
2,,,
3,,0034W00002NjFl5QAF,0034W00002NjFl5QAF
4,00Q4W00001RP3sPUAT,,00Q4W00001RP3sPUAT


In [7]:
hs_contact_campaign_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 207465 entries, 0 to 207464
Data columns (total 24 columns):
 #   Column                      Non-Null Count   Dtype 
---  ------                      --------------   ----- 
 0   id                          207465 non-null  int64 
 1   createdAt                   207465 non-null  object
 2   updatedAt                   207465 non-null  object
 3   archived                    207465 non-null  bool  
 4   campaign___paid             207005 non-null  object
 5   campaign_source             207005 non-null  object
 6   createdate                  207465 non-null  object
 7   hs_analytics_source         207465 non-null  object
 8   hs_analytics_source_data_1  207465 non-null  object
 9   hs_analytics_source_data_2  207431 non-null  object
 10  hs_object_id                207465 non-null  int64 
 11  lastmodifieddate            207465 non-null  object
 12  lead_source                 207181 non-null  object
 13  salesforcecontactid         2

In [8]:
# Output exploded version of hubspot contacts
hs_contact_campaign_df.to_csv("hubspot_contact_exploded.csv", index=False)

In [9]:
# hs_contact_campaign_df["createdAt"] = pd.to_datetime(hs_contact_campaign_df["createdAt"])
hs_contact_campaign_df["campaign.created_at"] = hs_contact_campaign_df["campaign_paid.timestamp"].map(datetime_parser.parse)
# hs_contact_campaign_df["createdAt"] = hs_contact_campaign_df["createdAt"].tz_localize(None)

In [10]:
hs_contact_campaign_df["campaign.created_at"].head()

0   2021-04-29 09:39:38.109000+00:00
1   2021-04-29 09:39:38.994000+00:00
2   2021-04-29 09:39:39.139000+00:00
3   2021-04-29 09:39:38.446000+00:00
4   2021-04-29 09:39:39.018000+00:00
Name: campaign.created_at, dtype: datetime64[ns, tzutc()]

In [11]:
hs_contact_campaign_df.shape

(207465, 25)

In [12]:
# Filter down data to 2020 and later
newer_hs_contact_campaign_df = hs_contact_campaign_df[
    hs_contact_campaign_df["campaign.created_at"] >= datetime(year=2020, month=1, day=1).isoformat()
]

In [13]:
newer_hs_contact_campaign_df.shape

(207465, 25)

In [14]:
newer_hs_contact_campaign_df.head()

,id,createdAt,updatedAt,archived,campaign___paid,campaign_source,createdate,hs_analytics_source,hs_analytics_source_data_1,hs_analytics_source_data_2,...,campaign_paid.value,campaign_paid.timestamp,campaign_paid.sourceType,campaign_paid.sourceId,campaign_source.value,campaign_source.timestamp,campaign_source.sourceType,campaign_source.sourceId,salesforce_id,campaign.created_at
0,5551,2017-09-26T08:28:27.856Z,2022-04-12T10:35:09.307Z,False,spotahome_hyper_growth,linkedin / impressions,2017-09-26T08:28:27.856Z,OTHER_CAMPAIGNS,spotahome_hyper_growth,linkedin / impressions,...,spotahome_hyper_growth,2021-04-29T09:39:38.109Z,AUTOMATION_PLATFORM,enrollmentId:101078377809;actionExecutionIndex:2,linkedin / impressions,2021-04-29T09:39:37.188Z,AUTOMATION_PLATFORM,enrollmentId:101078377809;actionExecutionIndex:1,00Q4W00001ROq8FUAT,2021-04-29 09:39:38.109000+00:00
1,5601,2017-09-26T09:13:41.779Z,2022-04-12T10:35:08.252Z,False,spotahome_hyper_growth,linkedin / impressions,2017-09-26T09:13:41.779Z,OTHER_CAMPAIGNS,spotahome_hyper_growth,linkedin / impressions,...,spotahome_hyper_growth,2021-04-29T09:39:38.994Z,AUTOMATION_PLATFORM,enrollmentId:101078377852;actionExecutionIndex:2,linkedin / impressions,2021-04-29T09:39:38.154Z,AUTOMATION_PLATFORM,enrollmentId:101078377852;actionExecutionIndex:1,00Q4W00001RP4oEUAT,2021-04-29 09:39:38.994000+00:00
2,10851,2017-09-27T14:45:36.035Z,2022-02-06T18:42:47.198Z,False,spotahome_hyper_growth,linkedin / impressions,2017-09-27T14:45:36.035Z,OTHER_CAMPAIGNS,spotahome_hyper_growth,linkedin / impressions,...,spotahome_hyper_growth,2021-04-29T09:39:39.139Z,AUTOMATION_PLATFORM,enrollmentId:101077198159;actionExecutionIndex:2,linkedin / impressions,2021-04-29T09:39:38.078Z,AUTOMATION_PLATFORM,enrollmentId:101077198159;actionExecutionIndex:1,,2021-04-29 09:39:39.139000+00:00
3,22551,2017-09-29T14:18:44.589Z,2022-04-12T10:35:07.355Z,False,spotahome_hyper_growth,linkedin / impressions,2017-09-29T14:18:44.589Z,OTHER_CAMPAIGNS,spotahome_hyper_growth,linkedin / impressions,...,spotahome_hyper_growth,2021-04-29T09:39:38.446Z,AUTOMATION_PLATFORM,enrollmentId:101074311528;actionExecutionIndex:2,linkedin / impressions,2021-04-29T09:39:37.184Z,AUTOMATION_PLATFORM,enrollmentId:101074311528;actionExecutionIndex:1,0034W00002NjFl5QAF,2021-04-29 09:39:38.446000+00:00
4,100801,2017-10-19T22:36:29.019Z,2022-04-12T10:35:07.576Z,False,goodlord case study,Google,2017-10-19T22:36:29.019Z,PAID_SEARCH,goodlord case study,goal setting,...,goodlord case study,2021-04-29T09:39:39.018Z,AUTOMATION_PLATFORM,enrollmentId:101078897924;actionExecutionIndex:3,Google,2021-04-29T09:39:38.397Z,AUTOMATION_PLATFORM,enrollmentId:101078897924;actionExecutionIndex:2,00Q4W00001RP3sPUAT,2021-04-29 09:39:39.018000+00:00


In [15]:
# Remove rows that don't have a value for campaign___paid
newer_hs_contact_campaign_df = newer_hs_contact_campaign_df[
    ~newer_hs_contact_campaign_df["campaign_paid.value"].isnull()
]

In [16]:
newer_hs_contact_campaign_df.shape

(207465, 25)

In [17]:
# linkedin_hs_contact_campaign_df = newer_hs_contact_campaign_df[
#     newer_hs_contact_campaign_df["campaign_source.value"].str.contains("linkedin", case=False)
# ]

In [18]:
# linkedin_hs_contact_campaign_df.head()

In [19]:
# linkedin_hs_contact_campaign_df.shape

In [20]:
# Merge existing df with the hubspot campaigns
combined_df = grow_df.merge(
    newer_hs_contact_campaign_df,
    how="left",
    left_on="campaign",
    right_on="campaign_paid.value"
)

combined_df.shape

(32454116, 34)

In [21]:
# Read in Salesforce data
sf_df = pd.read_csv("salesforce_prospects_data.csv")

sf_df.shape

C:\Users\tavon\AppData\Local\Temp\ipykernel_51472\2004071610.py:2: DtypeWarning: Columns (0,2,3,4,5,6) have mixed types. Specify dtype option on import or set low_memory=False.
  sf_df = pd.read_csv("salesforce_prospects_data.csv")


(288893, 38)

In [22]:
# Read in the MQL dates to determine whether a campaign became an MQL
sf_mql_dates_df = pd.read_csv("mql_dates.csv")

sf_mql_dates_df.shape

(289986, 2)

In [23]:
sf_mql_dates_df.head()

,Id,Became_a_marketing_qualified_lead_date__c
0,0034W00002NjEeyQAF,NaN
1,0034W00002NjEezQAF,NaN
2,0034W00002NjEf0QAF,2020-01-17 11:07:07
3,0034W00002NjEf1QAF,NaN
4,0034W00002NjEf2QAF,2020-01-22 05:44:06


In [24]:
# Join the two pieces of Salesforce data
combined_sf_df = sf_df.merge(
    sf_mql_dates_df,
    how="left",
    left_on="Id",
    right_on="Id"
)

combined_sf_df.shape

(288899, 39)

In [25]:
combined_sf_df.head()

,Gtmhub_Edition_ID__c,Status,IsConverted,ConvertedDate,ConvertedAccountId,ConvertedContactId,ConvertedOpportunityId,Became_a_marketing_qualified_lead_date__c_x,Id,AccountId,...,Total_Customer_MRR__c,new_business_count,partner_count,renewal_count,upsell_count,call_count,email_count,other_count,event_count,Became_a_marketing_qualified_lead_date__c_y
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0034W00002NjEeyQAF,0014W00002FhDWeQAN,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0034W00002NjEezQAF,0014W00002FhDOLQA3,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0034W00002NjEf0QAF,0014W00002Fh6yWQAR,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2020-01-17 11:07:07
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0034W00002NjEf1QAF,0014W00002Fh6yWQAR,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0034W00002NjEf2QAF,0014W00002Fh6yWQAR,...,1680.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2020-01-22 05:44:06


In [26]:
combined_df.columns.tolist()

['Date',
 'campaign',
 'Conversions',
 'Clicks',
 'Impressions',
 'Spend',
 'Sum of mqls',
 'Sum of In Trial',
 'Source',
 'id',
 'createdAt',
 'updatedAt',
 'archived',
 'campaign___paid',
 'campaign_source',
 'createdate',
 'hs_analytics_source',
 'hs_analytics_source_data_1',
 'hs_analytics_source_data_2',
 'hs_object_id',
 'lastmodifieddate',
 'lead_source',
 'salesforcecontactid',
 'salesforceleadid',
 'campaign_paid.value',
 'campaign_paid.timestamp',
 'campaign_paid.sourceType',
 'campaign_paid.sourceId',
 'campaign_source.value',
 'campaign_source.timestamp',
 'campaign_source.sourceType',
 'campaign_source.sourceId',
 'salesforce_id',
 'campaign.created_at']

In [27]:
combined_df.head()

,Date,campaign,Conversions,Clicks,Impressions,Spend,Sum of mqls,Sum of In Trial,Source,id,...,campaign_paid.value,campaign_paid.timestamp,campaign_paid.sourceType,campaign_paid.sourceId,campaign_source.value,campaign_source.timestamp,campaign_source.sourceType,campaign_source.sourceId,salesforce_id,campaign.created_at
0,2022-04-13,nurture-enterprise-positioning-v1-mof-rt-search,0.0,1.0,3.0,31.218599,NaN,NaN,Google Ads,163347401.0,...,nurture-enterprise-positioning-v1-mof-rt-search,2021-12-30T19:56:48.547Z,AUTOMATION_PLATFORM,enrollmentId:206519676287;actionExecutionIndex:3,Google,2021-12-30T19:56:47.548Z,AUTOMATION_PLATFORM,enrollmentId:206519676287;actionExecutionIndex:2,00Q4W00001ZYUnAUAX,2021-12-30 19:56:48.547000+00:00
1,2022-04-13,nurture-enterprise-positioning-v1-mof-rt-search,0.0,1.0,3.0,31.218599,NaN,NaN,Google Ads,187076001.0,...,nurture-enterprise-positioning-v1-mof-rt-search,2022-03-17T07:07:12.116Z,AUTOMATION_PLATFORM,enrollmentId:246091255180;actionExecutionIndex:3,Google,2022-03-17T07:07:11.128Z,AUTOMATION_PLATFORM,enrollmentId:246091255180;actionExecutionIndex:2,00Q8a00001p7yJEEAY,2022-03-17 07:07:12.116000+00:00
2,2022-04-13,nurture-enterprise-positioning-v1-mof-rt-search,0.0,1.0,3.0,31.218599,NaN,NaN,Google Ads,188414951.0,...,nurture-enterprise-positioning-v1-mof-rt-search,2022-03-24T17:59:01.383Z,AUTOMATION_PLATFORM,enrollmentId:250139668103;actionExecutionIndex:3,Google,2022-03-24T17:59:00.215Z,AUTOMATION_PLATFORM,enrollmentId:250139668103;actionExecutionIndex:2,00Q8a00001p86YOEAY,2022-03-24 17:59:01.383000+00:00
3,2022-04-13,lead-ei-idc-report-v1-tof-noam-search,0.0,0.0,101.0,0.000000,NaN,NaN,Google Ads,192222651.0,...,lead-ei-idc-report-v1-tof-noam-search,2022-04-02T23:10:58.438Z,AUTOMATION_PLATFORM,enrollmentId:254889772963;actionExecutionIndex:3,Google,2022-04-02T23:10:57.630Z,AUTOMATION_PLATFORM,enrollmentId:254889772963;actionExecutionIndex:2,00Q8a00001p8KFzEAM,2022-04-02 23:10:58.438000+00:00
4,2022-04-13,lead-ei-idc-report-v1-tof-noam-search,0.0,0.0,101.0,0.000000,NaN,NaN,Google Ads,193348901.0,...,lead-ei-idc-report-v1-tof-noam-search,2022-04-05T20:41:59.945Z,AUTOMATION_PLATFORM,enrollmentId:256327713072;actionExecutionIndex:3,Google,2022-04-05T20:41:59.252Z,AUTOMATION_PLATFORM,enrollmentId:256327713072;actionExecutionIndex:2,00Q8a00001q15g6EAA,2022-04-05 20:41:59.945000+00:00


In [28]:
# Fill NA
combined_df["salesforce_id"] = combined_df["salesforce_id"].fillna("")

In [29]:
# Join Salesforce data to the main dataset.
main_df = combined_df.merge(
    combined_sf_df,
    how="left",
    left_on="salesforce_id",
    right_on="Id"
)

main_df.shape

(32456129, 73)

In [30]:
output_path = os.path.join("grow_data.csv")
main_df.to_csv(output_path, index=False)

OSError: [Errno 28] No space left on device